In [ ]:
%load_ext autoreload
%autoreload 2

# Exercise 4

<img src="./images/04.png" width=800>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision 
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader

import os
import mlflow
from torchinfo import summary
from utils import train_network, accuracy_score_wrapper



/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_06/utils.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns06_4'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise06_4')

2025/06/08 15:29:31 INFO mlflow.tracking.fluent: Experiment with name 'Exercise06_2' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_06/mlruns06_2/743342736727177787', creation_time=1749383971503, experiment_id='743342736727177787', last_update_time=1749383971503, lifecycle_stage='active', name='Exercise06_2', tags={}>

In [2]:
torch.backends.cudnn.deterministic=True
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Dataset and Dataloader

In [4]:
train_data = torchvision.datasets.FashionMNIST("./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.FashionMNIST("./", train=True, transform=transforms.ToTensor(), download=True)

batch_size = 256
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [5]:
W = train_data.data.shape[1] 
H = train_data.data.shape[2]
D = W * H
C = 1
classes = len(torch.unique(train_data.targets))
n_filters = 32
hidden_neurons = 256

## Models

In [6]:
class ResidualBlockE(nn.Module):
    def __init__(self, in_featurs, out_featurs, activation=nn.LeakyReLU(.1)):
        super().__init__()
        self.F = nn.Sequential(
            nn.Linear(in_featurs, out_featurs),
            nn.BatchNorm1d(out_featurs),
            activation,
            nn.Linear(out_featurs, out_featurs),
            nn.BatchNorm1d(out_featurs),
            activation
        )
    
    def forward(self, x):
        return x + self.F(x)

In [7]:
def fc_bn(D, hidden_neurons, num_layers, classes):
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(D, hidden_neurons), nn.BatchNorm1d(hidden_neurons), nn.LeakyReLU(.1),
        *[nn.Sequential(
            nn.Linear(hidden_neurons, hidden_neurons),
            nn.BatchNorm1d(hidden_neurons),
            nn.LeakyReLU(.1))
            for _ in range(num_layers)],
        nn.Linear(hidden_neurons, classes)
        )

In [8]:
def fc_resblockE(D, hidden_neurons, num_layers, classes):
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(D, hidden_neurons), nn.BatchNorm1d(hidden_neurons), nn.LeakyReLU(.1),
        *[ResidualBlockE(hidden_neurons, hidden_neurons) for _ in range(num_layers//2)],
        nn.Linear(hidden_neurons, classes)
        )

## Training

In [ ]:
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 10
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

In [9]:
type_models = {
    'fc_bn': fc_bn,
    'fc_resblockE': fc_resblockE,
    }

In [ ]:
for experiment, fc_model in type_models.items():
    print('experiment: ', experiment)
    for num_layers in range(2, 21, 2):  
        print(f"num_layers: {num_layers} ")
        model = fc_model(D, hidden_neurons, num_layers, classes)
        optimizer = torch.optim.AdamW(model.parameters())
        params['optimizer'] = optimizer.defaults
        params['num_layers'] = num_layers
        params['experiment'] = experiment
        with open('model_summary.txt', 'w') as f:
            f.write(str(summary(model, input_size=(batch_size, C, W, H))))
        with mlflow.start_run(nested=True, run_name=experiment+f'{num_layers}'):
            mlflow.log_artifact('model_summary.txt')
            mlflow.log_params(params)
            fc_results = train_network(
                model=model,
                loss_func=loss_func,
                train_loader=train_loader,
                valid_loader=test_loader,
                # test_loader=test_loader,
                epochs=epochs,
                optimizer=optimizer,
                score_funcs=score_funcs,
                device=device,
                )

<img src="./images/E4_train_acc_selected.png">

<img src="./images/E4_train_loss_selected.png">

<img src="./images/E4_valid_acc_selected.png">

<img src="./images/E4_valid_loss_selected.png">

<img src="./images/E4_time_selected.png">